In [152]:
#install required packages
!pip install transformers

In [ ]:
#test GPU device
import tensorflow as tf
tf.test.gpu_device_name()
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7239110216727890324
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14328594432
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 337425258359147959
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [153]:
#add bert model
from transformers import BertTokenizerFast, BertForTokenClassification
from torch.nn.functional import softmax
import torch
#import packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import precision_score, recall_score


In [154]:
#import dataset
# test_df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
#import dataset from github
ADETrainingdata = pd.read_csv('https://raw.githubusercontent.com/FANMISUA/TweetAENormalization/main/ADEClassification/ADETraining.tsv', delimiter='\t')
ADEValidationdata = pd.read_csv('https://raw.githubusercontent.com/FANMISUA/TweetAENormalization/main/ADEClassification/ADEValidation.tsv', delimiter='\t')

In [155]:
#import normalization data
RawTweets = pd.read_csv('https://raw.githubusercontent.com/FANMISUA/TweetAENormalization/main/ADENormalization/Dev/tweets.tsv',delimiter='\t')
SpansNorm = pd.read_csv('https://raw.githubusercontent.com/FANMISUA/TweetAENormalization/main/ADENormalization/Dev/spans_norm.tsv',delimiter='\t')

#check the tweets id in Spans Norm and Raw Tweets
# print("get the tweets id in Raw Tweets:\n",RawTweets.iloc[:,0])
# print("Get the tweets id in Spans Norm:\n",SpansNorm.iloc[:,0])
list1 = RawTweets.iloc[:,0]
list2 = SpansNorm.iloc[:,0]
common_values = set(list1).intersection(set(list2))
print("common tweets id:",common_values)
print("Total tweets:",len(set(list1)),"Tweets with AE:",len(set(list2)),"matched tweets:",len(common_values))

common tweets id: {'SMM4H2022abr0OQs8okhO3F1M', 'SMM4H2022GEEGMyvixhP65MzB', 'SMM4H2022UGqZXi74Aq6rkF4J', 'SMM4H2022cjwbGQbnkpVjjJzR', 'SMM4H20221gMMy4c5pFbIYXpE', 'SMM4H2022zS5jwcGxhKiV9M6s', 'SMM4H2022HKMAb7UyA9DBgC0X', 'SMM4H2022RKHfXCWIZXEAIC6c', 'SMM4H20226b5WyZPTAJ7qL3dE', 'SMM4H2022MzDdtDqSq1LhsEKO', 'SMM4H2022UIXArsYjQ9fMwI4H', 'SMM4H2022UAvDTQWOIacvBkzp', 'SMM4H20221fZluiIo0whXaJA6', 'SMM4H2022WDJyq2OdnyWLMJT5', 'SMM4H2022Wh5BgCEutjAsILyO', 'SMM4H2022rZ604dAppHEgW0eE', 'SMM4H2022TIrTKWWfCAO02Uli', 'SMM4H202280H6GSvA1HFITRkC', 'SMM4H2022xPmAaH9Kvenuxhu2', 'SMM4H20223wRT70sBQURHsKEE', 'SMM4H2022TuXafFONznuqO8aQ', 'SMM4H2022nWG1t3YGOaL80SEc', 'SMM4H2022qpOud7DsjWMhWhFb', 'SMM4H2022PInYkYqdCaYoOMHW', 'SMM4H2022PHaTAtxs8MUXKgxV', 'SMM4H2022lX1A8RVuySkDzAB2', 'SMM4H2022eUHEdYa9HaG1N9d7', 'SMM4H20222sbwdYiQfnBCsPsl', 'SMM4H2022Dd6AFmou1yEhnoRH', 'SMM4H2022Xd3yVjFU568MhuMv', 'SMM4H2022yfpTy24FLthRUaay', 'SMM4H2022r2Rhx2VTef0t53fP', 'SMM4H2022w1Dip50oWfNYBVdo', 'SMM4H2022qZC2BPG2BW7UC1

In [ ]:
# from transformers import AutoTokenizer, AutoModelForTokenClassification

# # Load BioBERT model and tokenizer
# model_name = "monologg/biobert_v1.1_pubmed"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForTokenClassification.from_pretrained(model_name)

# # Example input text
# text = "The patient was prescribed paracetamol and ibuprofen for pain relief."

# # Tokenize the input text
# tokens = tokenizer.encode_plus(text, truncation=True, padding=True, return_tensors='pt')

# # Perform inference with BioBERT model
# with torch.no_grad():
#     outputs = model(**tokens)

# # Get the predicted labels
# predicted_labels = torch.argmax(outputs.logits, dim=2).squeeze()

# # Map the predicted labels to drug terms
# drugs = []
# for token, label in zip(tokens['input_ids'][0], predicted_labels):
#     if label != tokenizer.pad_token_id:
#         drugs.append(tokenizer.decode(token))

# # Print the extracted drugs
# print(drugs)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at monologg/biobert_v1.1_pubmed and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['##p', '[SEP]']


In [156]:
# train the ADE model
#print(ADETrainingdata.head() , ADEValidationdata.head())
#use the first 2000 rows of the training data
# ADETrainingdata[:2000]
# ADETrainingdata.iloc[:3,-3:]
#ADETrainingdata['tweet_id'][:200]
#only use the first 2000 tweets in Training data as training
print(ADETrainingdata['label'][:300].value_counts())
training_batch_1 =ADETrainingdata[:300]
print(training_batch_1.head())

0    255
1     45
Name: label, dtype: int64
              tweet_id                                               text  \
0  1382343793341575169  @IrvineWelsh I don’t know about you Irvine but...   
1  1377631738692796417  I bet money if i went n took a covid test righ...   
2  1386448010029240326  @JamesMelville My wife received a POSITIVE Cov...   
3  1361342676340211717  Out of the 180,000+ people who have had the tw...   
4  1386757983254765569  My whole family is sick af and here I am now i...   

   label  
0      0  
1      0  
2      0  
3      0  
4      0  


In [184]:
#balance the data
t_batch_ade = training_batch_1[training_batch_1['label'] == 1]
t_batch_noade = training_batch_1[training_batch_1['label'] == 0]
# print(t_batch_ade.shape,t_batch_noade.shape)
#downsampling
t_batch_noade_downsampled = t_batch_noade.sample(t_batch_ade.shape[0])
t_batch_balanced = pd.concat([t_batch_ade,t_batch_noade_downsampled])
# print(t_batch_balanced['label'].value_counts())
#proprecess the tweets
print("before clean",t_batch_balanced)
t_batch_balanced['text'] = proprocess_tweets(t_batch_balanced['text'])
print("after clean  ",t_batch_balanced)


before clean                 tweet_id                                               text  \
6    1383272654212272136  Test came back positive, no surprise. I have c...   
9    1362671045136809985  I’m going to sound like I have lost my marbles...   
10   1373077971658022918  I just tested positive for Covid-19 from two s...   
13   1379116535726415874  As I was in the ER last night I overheard a co...   
22   1375292877195120640  @Tempid_ I actually tested positive for covid....   
..                   ...                                                ...   
61   1382471005210341383  @DoorDash as if you guys can’t get any worse. ...   
151  1380919039061602305  @OwenJon29550837 @djenkinfitness @UKCovid19Sta...   
281  1377037200349569024  @nathanfletcher I see so far we have 70 positi...   
93   1383623674901663747  @thomaskaine5 @proudCanadavet My sis in law ju...   
20   1333413081083310081  There are Lies, Damn lies, and Statistics.http...   

     label  
6        1  
9        1  

In [159]:
#split training and testing
train_x, test_x, train_y, test_y = train_test_split(t_batch_balanced['text'], t_batch_balanced['label'],  test_size=0.2, random_state=42)
# test_x.shape


(18,)

In [ ]:
#add preprocessing step to clean the tweets
import re
def preprocessTweet1(tweet):
    # process the tweets

    # Remove 'RT' from tweet
    tweet = re.sub(r'RT[\s]+', '', tweet)
    # Remove &amp; (ampersand)
    tweet = re.sub(r'&amp;', 'and', tweet)
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet)
    #Convert @username to @USER
    tweet = re.sub('@[^\s]+','',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    # Remove all emojis.
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

    tweet = emoji_pattern.sub(r'', tweet)

    # Remove redundant spaces
    tweet = re.sub(r'\s+', ' ', tweet).strip()

    #remove unknown characters
    tweet = ''.join([c for c in tweet if ord(c) < 128])

    #trim
    tweet = tweet.strip('\'"')
    return tweet



In [ ]:
def proprocess_tweets(tweets):
  processed_tweets = [preprocessTweet1(tweet) for tweet in tweets]
  return processed_tweets

In [178]:
print(t_batch_balanced['text'])
print(proprocess_tweets(t_batch_balanced['text']))


6      Test came back positive, no surprise. I have c...
9      I’m going to sound like I have lost my marbles...
10     I just tested positive for Covid-19 from two s...
13     As I was in the ER last night I overheard a co...
22     @Tempid_ I actually tested positive for covid....
                             ...                        
75     I will not get the Pfizer vaccine I pfeel just...
262    @jashby23 I would get it if he had COVID or go...
182    What’s the point of getting vaccinated if you’...
246    A small milestone. It’s been over a year since...
238    I found out one of my family members got covid...
Name: text, Length: 90, dtype: object
["Test came back positive, no surprise. I have covid. Have been fully vaccinated since end of Jan, but the last 2 weeks at work have been on the Covid unit with patients that don't keep their masks on and have not the greatest hygiene, and myself not following strict PPE guidelines", 'Im going to sound like I have lost my marbles but

In [161]:
# def get_text_embeding(sentences):
#   tokenized = sentences['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
#   #load BERT model
#   #Testing small bert model 'distilbert-base-uncased'
#   model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
#   #testing pubmed bert model
#   # model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'ml4pubmed/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext_pub_section')

#   tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
#   model = model_class.from_pretrained(pretrained_weights)
#   return tokenized

In [164]:
# get_text_embeding(t_batch_balanced['text'])
t_batch_balanced['text']


6      Test came back positive, no surprise. I have c...
9      I’m going to sound like I have lost my marbles...
10     I just tested positive for Covid-19 from two s...
13     As I was in the ER last night I overheard a co...
22     @Tempid_ I actually tested positive for covid....
                             ...                        
75     I will not get the Pfizer vaccine I pfeel just...
262    @jashby23 I would get it if he had COVID or go...
182    What’s the point of getting vaccinated if you’...
246    A small milestone. It’s been over a year since...
238    I found out one of my family members got covid...
Name: text, Length: 90, dtype: object

In [165]:
# #test preprocessing

# tweet_test = 'I am officially over Long Covid, no more stays in the ER, lockdown restrictions are being lifted. ðŸ™ ðŸ˜€  @Mariska @warrenleightTV https://t.co/rJWreLHS3A'

# input_test = training_batch_1['text'][1]
# clean_tweet_test=preprocessTweet1(tweet_test)

# print("Input Tweet: ",training_batch_1['text'][0])

# print("Clean Tweet: ",preprocessTweet1(training_batch_1['text'][0]))

# # print("Raw tweet:",tweet_test)
# # print("Clean tweet:",clean_tweet_test)

Input Tweet:  @IrvineWelsh I don’t know about you Irvine but i keep being told that Covid doesn’t exist, therefore the doctors and nurses in hospital are scratching their arses looking for something to do. Brilliant hoax. The scientists are great actors and empty coffins are filling the cemeteries. Sillyme
Clean Tweet:  I dont know about you Irvine but i keep being told that Covid doesnt exist, therefore the doctors and nurses in hospital are scratching their arses looking for something to do. Brilliant hoax. The scientists are great actors and empty coffins are filling the cemeteries. Sillyme


In [166]:
#load BERT model
#Testing small bert model 'distilbert-base-uncased'
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
#testing pubmed bert model
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'ml4pubmed/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext_pub_section')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [186]:
#tokenize the training tweets
# tokenized = t_batch_balanced['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized = t_batch_balanced['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
# print("Input Tweet after cleaning: ",t_batch_balanced['text'])
print(tokenized)

6      [101, 3231, 2234, 2067, 3893, 1010, 2053, 4474...
9      [101, 10047, 2183, 2000, 2614, 2066, 1045, 203...
10     [101, 1045, 2074, 7718, 3893, 2005, 2522, 1725...
13     [101, 2004, 1045, 2001, 1999, 1996, 9413, 2197...
22     [101, 1045, 2941, 7718, 3893, 2005, 2522, 1725...
                             ...                        
61     [101, 2004, 2065, 2017, 4364, 2064, 2102, 2131...
151    [101, 2009, 2071, 2022, 2008, 2216, 2111, 7718...
281    [101, 1045, 2156, 2061, 2521, 2057, 2031, 3963...
93     [101, 2026, 24761, 1999, 2375, 2074, 2288, 491...
20     [101, 2045, 2024, 3658, 1010, 4365, 3658, 1010...
Name: text, Length: 90, dtype: object


In [191]:
#padding
#each tweet is represented as a list of tokens.
#add padding to make all the sentence to the max length so that BERT could process all at once.
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
print("max len for padding: ",max_len, np.array(padded).shape)


max len for padding:  83 (90, 83)


In [192]:
#Masking the paddding
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape
# print(attention_mask[10])

(90, 83)

In [193]:
#Use pretrained Bert model to word embedding all the sentences
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)



In [194]:
#Get the tensor dimension
print(last_hidden_states[0])
print(last_hidden_states[0].size())

tensor([[[-0.0077, -0.0207,  0.1494,  ..., -0.0705,  0.4320,  0.3934],
         [ 0.3990, -0.2975,  0.2578,  ..., -0.4934,  0.4676, -0.4215],
         [-0.2777, -0.4936,  0.7285,  ..., -0.2239,  0.3164, -0.0129],
         ...,
         [ 0.0784,  0.0143,  0.0455,  ..., -0.3455,  0.0210, -0.3327],
         [-0.0151,  0.0204,  0.0850,  ..., -0.3674,  0.0930, -0.2946],
         [ 0.0107, -0.0567,  0.0802,  ..., -0.3056, -0.0385, -0.0550]],

        [[ 0.1890,  0.1106,  0.0343,  ...,  0.0940,  0.5146,  0.6049],
         [-0.2584,  0.4436,  0.7963,  ...,  0.0244,  0.3090,  0.7690],
         [ 0.4610, -0.3986,  0.2908,  ...,  0.3469, -0.1381, -0.0023],
         ...,
         [ 0.4575,  0.0775,  0.7056,  ...,  0.0737, -0.1037,  0.2346],
         [ 0.2644, -0.0421,  0.7823,  ..., -0.0911, -0.1165,  0.2872],
         [ 0.2944, -0.0641,  0.4032,  ..., -0.0370, -0.0714, -0.1064]],

        [[ 0.0788, -0.0436,  0.1540,  ...,  0.0353,  0.4016,  0.3319],
         [ 0.6385, -0.3469,  0.1724,  ...,  0

In [ ]:
# #testing cos similarity for the encoding
# testData = {
#     "id": [1, 2, 3, 4],
#     "text": [
#         "banana.",
#         "grape.",
#         "bill gates.",
#         "elon mask."    ]
# }
# tD = pd.DataFrame(testData)
# print(tD['text'])

# # tokenize
# tokenized = tD['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
# print("Input Tweet: ",tD['text'][0])
# print("Clean Tweet: ",preprocessTweet1(tD['text'][0]))
# print("tokenize the tweet:",tokenized[0])
# #padding
# #each tweet is represented as a list of tokens. add padding to make all the sentence to the max length so that BERT could process all at once.
# max_len = 0
# for i in tokenized.values:
#     if len(i) > max_len:
#         max_len = len(i)

# padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
# # print(max_len, np.array(padded).shape)
# #Masking the paddding
# attention_mask = np.where(padded != 0, 1, 0)
# attention_mask.shape
# # print(padded)
# #Use pretrained Bert model to word embedding all the sentences
# input_ids = torch.tensor(padded)
# attention_mask = torch.tensor(attention_mask)

# with torch.no_grad():
#     last_hidden_states = model(input_ids, attention_mask=attention_mask)

# #Get the tensor dimension
# # print(last_hidden_states[0])
# print(last_hidden_states[0].size())
# from sklearn.metrics.pairwise import cosine_similarity
# fea = last_hidden_states[0][:,0,:].numpy()
# fea
# # newfea =np.array[fea]
# # print(cosine_similarity(newfea[1],newfea[2]))
# cosine_similarity([fea[3]],[fea[0]])

0        banana.
1         grape.
2    bill gates.
3     elon mask.
Name: text, dtype: object
Input Tweet:  banana.
Clean Tweet:  banana.
tokenize the tweet: [101, 15212, 1012, 102]
torch.Size([4, 6, 768])


array([[0.9582989]], dtype=float32)

In [ ]:
# #padding
# #each tweet is represented as a list of tokens. add padding to make all the sentence to the max length so that BERT could process all at once.
# max_len = 0
# for i in tokenized.values:
#     if len(i) > max_len:
#         max_len = len(i)

# padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
# print(max_len, np.array(padded).shape)
# print(padded[11])

283 (300, 283)
[  101  2619  1045  2293  2200  2172  2001 11441  2007  2522 17258  2539
  2651  1012  3531  4929 15806  1010  9378  2115  2398  1998  2994  1996
 17752  2185  2013  2169  2060  1012  3531  1012  2057  2024  2471  2045
  1012   102     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0

In [ ]:
# #Masking the paddding
# attention_mask = np.where(padded != 0, 1, 0)
# attention_mask.shape
# print(attention_mask[10])

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
# #Use pretrained Bert model to word embedding all the sentences
# input_ids = torch.tensor(padded)
# attention_mask = torch.tensor(attention_mask)

# with torch.no_grad():
#     last_hidden_states = model(input_ids, attention_mask=attention_mask)



In [ ]:
# #Get the tensor dimension
# print(last_hidden_states[0])
# print(last_hidden_states[0].size())
# #first position is [CLS] token.
# features = last_hidden_states[0][:,0,:].numpy()
# print(features)
# print(features.shape)

tensor([[[ 0.2829,  0.1567,  0.2272,  ..., -0.1869,  0.6070,  0.3700],
         [ 0.6401, -0.0036,  0.4816,  ...,  0.0065,  0.3466,  0.0479],
         [ 0.0592, -0.1083,  0.4402,  ..., -0.3974, -0.1379,  0.2461],
         ...,
         [ 0.5154,  0.2809,  0.0904,  ..., -0.2032,  0.3637,  0.0181],
         [ 0.5656,  0.1848,  0.1909,  ..., -0.0981,  0.3749, -0.2579],
         [ 0.4555,  0.0392,  0.1548,  ...,  0.0069,  0.2053, -0.0969]],

        [[ 0.0924,  0.1797,  0.0927,  ..., -0.1305,  0.4414,  0.4691],
         [ 0.4584,  0.1091,  0.2552,  ..., -0.0335,  0.4750,  0.5256],
         [ 1.0446, -0.2030,  0.2586,  ..., -0.5464,  0.1183,  0.2701],
         ...,
         [ 0.1503,  0.1455,  0.2511,  ..., -0.0866, -0.1659,  0.2501],
         [-0.0349,  0.2445,  0.2170,  ..., -0.1941,  0.0664,  0.1888],
         [ 0.1751,  0.0580,  0.3602,  ..., -0.0849, -0.0144,  0.2471]],

        [[-0.0798, -0.2039,  0.0721,  ..., -0.2190,  0.3414,  0.4212],
         [ 0.4364, -0.1264,  0.3858,  ...,  0

In [218]:
#first position is [CLS] token.
#convert 3D tensor to 2D tensor
tensor_3d = last_hidden_states[0]

# Convert 3D tensor to 2D tensor
tensor_2d = tensor_3d.reshape(tensor_3d.shape[0], -1).numpy()

print("Original 3D tensor shape:", tensor_3d.shape)
print("Converted 2D tensor shape:", tensor_2d.shape)
# print(tensor_2d)
features = tensor_2d
print("features dimension:",features.shape)

#label indicating which sentence is positive and negative.
labels = t_batch_balanced['label']
# print(labels)
print("label dimension:",labels.shape)

#use classification models

train_features, test_features, train_labels, test_labels = train_test_split(features, labels,  test_size=0.2, random_state=42)
print("training feature dimension",train_features.shape)
print("testing feature dimension",test_features.shape)
print("training label dimension",train_labels.shape)
print("training label dimension",test_labels.shape)


Original 3D tensor shape: torch.Size([90, 83, 768])
Converted 2D tensor shape: (90, 63744)
features dimension: (90, 63744)
label dimension: (90,)
training feature dimension (72, 63744)
testing feature dimension (18, 63744)
training label dimension (72,)
training label dimension (18,)


In [ ]:
# #split the training and testing data
# #240 tweets as training; 60 tweets as testing
# train_features, test_features, train_labels, test_labels = train_test_split(features, labels,  test_size=0.2, random_state=42)
# print(train_features.shape)
# print(test_features.shape)
# print(train_labels.shape)
# print(test_labels.shape)


(240, 768)
(60, 768)
(240,)
(60,)


In [219]:
#refine parameters for logisticRegression model
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 5.263252631578947}
best scrores:  0.7219047619047619


In [220]:
lr_clf = LogisticRegression(C=5.2)
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=5.2)

In [221]:
lr_clf.score(test_features, test_labels)

0.8888888888888888

In [222]:
lr_clf.score(train_features, train_labels)

1.0

In [223]:
from sklearn.metrics import precision_recall_curve, accuracy_score, f1_score

predict_test_labels = lr_clf.predict(test_features)
precision = precision_score(test_labels, predict_test_labels)
recall = recall_score(test_labels, predict_test_labels)
accuracy =accuracy_score (test_labels, predict_test_labels)
f1 = f1_score(test_labels, predict_test_labels)


print("Precision: ", precision)
print("Recall: ", recall)
print("accuracy:",accuracy)
print("F1:",accuracy)

precision, recall , thresholds = precision_recall_curve(test_labels, predict_test_labels)
thresholds

Precision:  1.0
Recall:  0.8333333333333334
accuracy: 0.8888888888888888
F1: 0.8888888888888888


array([0, 1])

In [242]:
# print(test_labels.shape)
# print(predict_test_labels.shape)
tp = np.sum((predict_test_labels == 1) & (test_labels == 1))
tn = np.sum((predict_test_labels == 0) & (test_labels == 0))
fp = np.sum((predict_test_labels == 0) & (test_labels == 1))
fn = np.sum((predict_test_labels == 1) & (test_labels == 0))


print(test_labels)
print(predict_test_labels)
print("tp:",tp,"tn:",tn,"fp:",fp,"fn:",fn)

282    1
148    1
124    0
31     0
6      1
187    1
280    1
63     0
60     1
299    1
201    0
241    1
145    0
151    0
70     1
22     1
121    1
197    1
Name: label, dtype: int64
[1 0 0 0 1 1 0 0 1 1 0 1 0 0 1 1 1 1]
tp: 10 tn: 6 fp: 2 fn: 0


In [247]:
#test code with neural network
import torch
from transformers import BertTokenizer, BertModel
import torch.nn as nn

# Load pre-trained BERT model and tokenizer
bert_model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Input text
text = "Hello, how are you?"

# Tokenize and encode text
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
input_ids = inputs['input_ids']

# Get BERT embeddings
with torch.no_grad():
    bert_output = bert_model(input_ids)

# Extract embeddings
bert_embeddings = bert_output.last_hidden_state
print(bert_embeddings.shape)
tensor_3dbert = bert_embeddings

# Convert 3D tensor to 2D tensor
tensor_2dbert = tensor_3dbert.reshape(tensor_3dbert.shape[0], -1).numpy()
print(tensor_2dbert.shape)
# Define a simple neural network
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

# Instantiate the neural network
input_size = bert_embeddings.shape[2]  # BERT embedding size
hidden_size = 64
output_size = 2  # Example: binary classification

net = NeuralNetwork(input_size, hidden_size, output_size)

# Use BERT embeddings as input to the neural network
output = net(bert_embeddings)
print(output.shape)


torch.Size([1, 8, 768])
(1, 6144)
torch.Size([1, 8, 2])


In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.846 (+/- 0.02)


In [ ]:
from transformers import pipeline
print(pipeline('sentiment-analysis')('I hate you'))


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'NEGATIVE', 'score': 0.9991129040718079}]
